In [1]:
#Importing all required Modules

from sqlalchemy.engine import create_engine
import pandas as pd
import datetime
from sqlalchemy.dialects.oracle import DATE, VARCHAR2, FLOAT, NUMBER,TIMESTAMP
from sqlalchemy import text,String,types
from datetime import datetime

In [13]:
# Extracting data from Source

with source_engine.begin():
    try:
        query = ('select distinct facility_name from AceExtraction.Medrite_input_table')
        clinic_names_df = pd.read_sql(query, source_engine)
        clinic_names = list(clinic_names_df['facility_name'])
    except Exception as e:
        print('Error: ',e)

In [14]:
#creating empty dataframe

clinic_dim_df = pd.DataFrame(columns=['clinic_dim_id', 'clinic_source_id', 'clinic_name', 'clinic_type','clinic_size','clinic_operational_since',
                                     'clinic_region','client_name','created_date'])

#insurance_firms_dim_df

In [15]:
#Loading the data into dataframe
clinic_dim_df['clinic_name'] = clinic_names

In [17]:
#Defining the data types

data_types = {'clinic_dim_id': types.Integer(),
              'clinic_source_id':types.String(255),
              'clinic_name':types.String(255),
              'clinic_type':types.String(255),
              'clinic_size':types.String(255),
              'clinic_operational_since':types.Date(),
              'clinic_region':types.String(255),
              'client_name':types.String(255),
              'created_date':types.Date() }


In [18]:
try:
 clinic_dim_df.to_sql('clinic_dim',staging_engine, if_exists='append', index=False, dtype= data_types)
 print('successfully loaded into Staging_DB')
except Exception as e:
 print("An error occurred:", e)

successfully loaded into Staging_DB


##ETL from staging to DataWarehouse

In [21]:
#Extracting data from staging DB

with staging_engine.begin():
 try:
     query = ("select distinct clinic_name from staging_source.clinic_dim;")
     staging_clinic_names_df = pd.read_sql_query(query, staging_engine)
     staging_clinic_names_df.dropna(subset=['clinic_name'], inplace=True)
     staging_clinic_names = list(staging_clinic_names_df['clinic_name'])
 except Exception as e:
     print("An error occurred:", e)

In [22]:
#creating empty dataframe

clinic_dim_df = pd.DataFrame(columns=['clinic_dim_id', 'clinic_source_id', 'clinic_name', 'clinic_type','clinic_size','clinic_operational_since',
                                     'clinic_region','client_name','created_date'])

#insurance_firms_dim_df

In [25]:
#Determining the current maximum ID in the DataFrame, 
current_max_id = clinic_dim_df['clinic_dim_id'].max() if not clinic_dim_df.empty else 0
# Using pandas Series to generate IDs
new_ids = pd.Series(range(current_max_id + 1, current_max_id + 1 + len(staging_clinic_names)))


In [27]:
#Transformation

new_records = pd.DataFrame({'clinic_dim_id': new_ids,
              'clinic_source_id':[category[:3].upper() + str('_') + str(new_id) for category, new_id in zip(staging_clinic_names, new_ids)],
              'clinic_name':staging_clinic_names,
              'clinic_type':'Unknown',
              'clinic_size':'Unknown',
              'clinic_operational_since':'Unknown',
              'clinic_region':'Unknown',
              'client_name':'Unknown',
              'created_date':(datetime.now().strftime('%Y-%m-%d'))})

In [29]:
# Appending the new records to the original DataFrame

clinic_dim_df = pd.concat([clinic_dim_df, new_records], ignore_index=True)

In [30]:
#Defining the data types

data_types = {'clinic_dim_id': types.Integer(),
              'clinic_source_id':types.String(255),
              'clinic_name':types.String(255),
              'clinic_type':types.String(255),
              'clinic_size':types.String(255),
              'clinic_operational_since':types.String(255),
              'clinic_region':types.String(255),
              'client_name':types.String(255),
              'created_date':types.Date() }


In [31]:
#Loading the data into Datawarehouse

try:
 clinic_dim_df.to_sql('clinic_dim',datawarehouse_engine, if_exists='append', index=False, dtype= data_types)
 print('successfully loaded into DataWarehouse')
except Exception as e:
 print("An error occurred:", e)

successfully loaded into DataWarehouse
